Import all the required headers

In [1]:
import theano
from theano import tensor as T
from theano.tensor.nnet import conv
import pandas as pd
import numpy as np
import cPickle 
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

Using gpu device 0: GeForce GT 750M (CNMeM is disabled)


Load data set for facial features

In [2]:
rng = np.random.RandomState(23455)
#load Facial expression dataset
dataset = pd.read_csv('fer2013.csv')
#print dataset.values
# instantiate 4D tensor for input
input = T.tensor4(name='input')
srng = RandomStreams()
#print dataset.values[0]
#print np.__path__




Helper Functions

In [18]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01) , borrow=True)

def rectify(X):
    return T.maximum(X, 0.)

def softmax(X):
    return T.nnet.softmax(X)

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def oneHotExpression(num):
    Y = np.zeros((1,7), dtype=np.int)
    Y[0][num]= 1;
    return Y

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

Model

In [19]:
def model(X, w, w2, w3, w4, p_drop_conv, p_drop_hidden):
    l1a = rectify(conv2d(X, w, border_mode='full'))
    l1 = max_pool_2d(l1a, (2, 2))
    l1 = dropout(l1, p_drop_conv)

    l2a = rectify(conv2d(l1, w2))
    l2 = max_pool_2d(l2a, (2, 2))
    l2 = dropout(l2, p_drop_conv)

    l3a = rectify(conv2d(l2, w3))
    l3b = max_pool_2d(l3a, (2, 2))
    l3 = T.flatten(l3b, outdim=2)
    l3 = dropout(l3, p_drop_conv)

    l4 = rectify(T.dot(l3, w4))
    l4 = dropout(l4, p_drop_hidden)

    pyx = softmax(T.dot(l4, w_o))
    return l1 , l2 , l3 , l4, pyx

Define symbolic expression of training and optimisation step

In [20]:
X = T.ftensor4('X')
Y = T.fmatrix('Y')

w = init_weights((32, 1, 3, 3))
w2 = init_weights((64, 32, 3, 3))
w3 = init_weights((128, 64, 3, 3))
w4 = init_weights((128 * 3 * 3, 625))
w_o = init_weights((625, 10))

noise_l1, noise_l2, noise_l3, noise_l4, noise_py_x = model(X, w, w2, w3, w4, 0.2, 0.5)
l1, l2, l3, l4, py_x = model(X, w, w2, w3, w4, 0., 0.)
y_x = T.argmax(py_x, axis=1)


cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [w, w2, w3, w4, w_o]
updates = RMSprop(cost, params, lr=0.001)

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)
#print np.asarray(dataset.values[i][0])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: max_pool_2d() will have the parameter ignore_border default value changed to True (currently False). To have consistent behavior with all Theano version, explicitly add the parameter ignore_border=True. On the GPU, using ignore_border=False is needed to use CuDNN. When using ignore_border=False and not using CuDNN, the only GPU combination supported is when `ds == st and padding == (0, 0) and mode == 'max'`. Otherwise, the convolution will be executed on CPU.
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: max_pool_2d() will have the parameter ignore_border default value changed to True (currently False). To have consistent behavior with all Theano version, explicitly add the parameter ignore_border=True. On the GPU, using ignore_border=False is needed to use CuDNN. When using 

Train network

In [21]:
for i in range(3000):
    trX = np.asarray(dataset.values[i][1].split(" ") , dtype="float64")
    trX = trX.reshape(-1, 1, 48, 48)
    trY = np.asarray(dataset.values[i][0])
    trY = oneHotExpression(trY)
    cost = train(trX, trY)
#print np.mean(np.argmax(teY, axis=1) == predict(teX))


RuntimeError: GpuCorrMM encountered a CUDA error in im2col: unspecified launch failure
This could be a known bug in CUDA, please see the GpuCorrMM() documentation.

Apply node that caused the error: GpuCorrMM{valid, (1, 1)}(GpuContiguous.0, GpuContiguous.0)
Toposort index: 73
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D)]
Inputs shapes: [(1, 32, 25, 25), (64, 32, 3, 3)]
Inputs strides: [(0, 625, 25, 1), (288, 9, 3, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuElemwise{maximum,no_inplace}(GpuCorrMM{valid, (1, 1)}.0, CudaNdarrayConstant{error while transferring the value: error (unspecified launch failure)copying data to host}), GpuElemwise{Composite{Cast{float32}(EQ(i0, i1))}}[(0, 0)](GpuElemwise{maximum,no_inplace}.0, GpuCorrMM{valid, (1, 1)}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. 